In [1]:
# pip install spotipy 

# Imports

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import time 

# Credentials

In [3]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group lol 
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

# Data Collection 

## Testing 20 Songs in Liked 

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'user-library-read playlist-read-private playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    count = 0
    total_liked_songs = results['total']
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
        count += 1
    print(f"Showing {count} out of {total_liked_songs} songs")
else:
    print("Can't get token for", username)

SexyBack (feat. Timbaland) - Justin Timberlake
Closer (feat. Halsey) - The Chainsmokers
Otra vez (feat. J Balvin) - Zion & Lennox
Robarte un Beso - Carlos Vives
I Love It (feat. Charli XCX) - Original Version - Icona Pop
Single Ladies (Put a Ring on It) - Beyoncé
Can't Feel My Face - The Weeknd
Want to Want Me - Jason Derulo
1, 2, 3 (feat. Jason Derulo & De La Ghetto) - Sofia Reyes
Worth It - Fifth Harmony
Dile la Verdad - The Rudeboyz
Fancy - Iggy Azalea
Mami - Piso 21
Side To Side - Ariana Grande
That's What I Like - Bruno Mars
7 rings - Ariana Grande
Happier - Marshmello
Baila Baila Baila - Ozuna
Cheap Thrills - Sia
Can't Hold Us - feat. Ray Dalton - Macklemore & Ryan Lewis
Showing 20 out of 973 songs


In [5]:
# Test
if results: 
    print("something")
else:
    print("nothing")

something


# Collecting Audio Features for Liked Songs

In [6]:
song_name = []
song_uri = []
song_popularity = []
artist_name = []
artist_uri = []
genres_found = []
loops= 0 
track_audio_features = []
while results:
    for item in results['items']:
        track = item['track']
        song_name.append(track['name'])
        song_uri.append(track['uri'])
        song_popularity.append(track['popularity'])
        #Some songs have multuple artists so I'll make a list of artists and a list of uri for each song
        artist_name.append([artist['name'] for artist in track['artists']])
        artist_uri.append([artist['uri'] for artist in track['artists']])
        temp = sp.artists([artist['uri'] for artist in track['artists']])['artists']
        temp2 = []
        for temp_artists in temp:
            for genre in temp_artists['genres']:
                temp2.append(genre)
        genres_found.append(list(set(temp2)))
    track_audio_features.extend(sp.audio_features(song_uri[20*loops :20*(loops+1)]))
    print(f'Audio Features were gathered for {len(track_audio_features)} songs')

    loops += 1 
    results = sp.next(results)
    if (loops % 5) == 0 :
        print(f'{loops} cycles have been completed')
    time.sleep(.8)
df_audio_feat = pd.DataFrame(track_audio_features) 
df_tracks = pd.DataFrame({'artist_uri': artist_uri,
                          'song_uri': song_uri,
                          'song_name':song_name,
                          'artists': artist_name,
                          'genres': genres_found,
                          'popularity': song_popularity})
data = pd.merge(df_tracks, df_audio_feat, left_on= 'song_uri', right_on= 'uri')
                          

Audio Features were gathered for 20 songs
Audio Features were gathered for 40 songs
Audio Features were gathered for 60 songs
Audio Features were gathered for 80 songs
Audio Features were gathered for 100 songs
5 cycles have been completed
Audio Features were gathered for 120 songs
Audio Features were gathered for 140 songs
Audio Features were gathered for 160 songs
Audio Features were gathered for 180 songs
Audio Features were gathered for 200 songs
10 cycles have been completed
Audio Features were gathered for 220 songs
Audio Features were gathered for 240 songs
Audio Features were gathered for 260 songs
Audio Features were gathered for 280 songs
Audio Features were gathered for 300 songs
15 cycles have been completed
Audio Features were gathered for 320 songs
Audio Features were gathered for 340 songs
Audio Features were gathered for 360 songs
Audio Features were gathered for 380 songs
Audio Features were gathered for 400 songs
20 cycles have been completed
Audio Features were gathe

In [7]:
data

,artist_uri,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"[spotify:artist:31TPClRtHm23RisEBtV3X7, spotif...",spotify:track:0O45fw2L5vsWpdsOdXwNAR,SexyBack (feat. Timbaland),"[Justin Timberlake, Timbaland]","[urban contemporary, hip pop, hip hop, dance p...",74,0.967,0.583,7,-5.562,...,0.0519,0.964,117.000,audio_features,0O45fw2L5vsWpdsOdXwNAR,spotify:track:0O45fw2L5vsWpdsOdXwNAR,https://api.spotify.com/v1/tracks/0O45fw2L5vsW...,https://api.spotify.com/v1/audio-analysis/0O45...,242733,4
1,"[spotify:artist:69GGBxA162lTqCwzJG5jLp, spotif...",spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,Closer (feat. Halsey),"[The Chainsmokers, Halsey]","[dance pop, post-teen pop, tropical house, ind...",84,0.748,0.524,8,-5.599,...,0.1110,0.661,95.010,audio_features,7BKLCZ1jbUBVqRi2FVlTVw,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,https://api.spotify.com/v1/audio-analysis/7BKL...,244960,4
2,"[spotify:artist:21451j1KhjAiaYKflxBjr1, spotif...",spotify:track:3QwBODjSEzelZyVjxPOHdq,Otra vez (feat. J Balvin),"[Zion & Lennox, J Balvin]","[reggaeton, latin hip hop, latin, reggaeton fl...",73,0.832,0.772,10,-5.429,...,0.4400,0.704,96.016,audio_features,3QwBODjSEzelZyVjxPOHdq,spotify:track:3QwBODjSEzelZyVjxPOHdq,https://api.spotify.com/v1/tracks/3QwBODjSEzel...,https://api.spotify.com/v1/audio-analysis/3QwB...,209453,4
3,"[spotify:artist:4vhNDa5ycK0ST968ek7kRr, spotif...",spotify:track:4z3GJkrtH97Bj6fRta983T,Robarte un Beso,"[Carlos Vives, Sebastian Yatra]","[reggaeton, champeta, latin, latin pop, vallen...",10,0.748,0.884,10,-3.556,...,0.3610,0.678,95.046,audio_features,4z3GJkrtH97Bj6fRta983T,spotify:track:4z3GJkrtH97Bj6fRta983T,https://api.spotify.com/v1/tracks/4z3GJkrtH97B...,https://api.spotify.com/v1/audio-analysis/4z3G...,193159,4
4,"[spotify:artist:1VBflYyxBhnDc9uVib98rw, spotif...",spotify:track:3VZQshi4COChhXaz7cLP02,I Love It (feat. Charli XCX) - Original Version,"[Icona Pop, Charli XCX]","[swedish electropop, swedish pop, dance pop, c...",64,0.714,0.905,8,-2.663,...,0.1460,0.838,125.927,audio_features,3VZQshi4COChhXaz7cLP02,spotify:track:3VZQshi4COChhXaz7cLP02,https://api.spotify.com/v1/tracks/3VZQshi4COCh...,https://api.spotify.com/v1/audio-analysis/3VZQ...,155337,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,[spotify:artist:1qto4hHid1P71emI6Fd8xi],spotify:track:0HDHY6RSHHG2ZTE0cMT4GJ,Los Infieles,[Aventura],"[latin hip hop, latin, bachata, latin pop, tro...",70,0.745,0.716,9,-8.221,...,0.0590,0.817,132.932,audio_features,0HDHY6RSHHG2ZTE0cMT4GJ,spotify:track:0HDHY6RSHHG2ZTE0cMT4GJ,https://api.spotify.com/v1/tracks/0HDHY6RSHHG2...,https://api.spotify.com/v1/audio-analysis/0HDH...,257187,4
969,[spotify:artist:47bVt95bvBMpmJFWoyhH0C],spotify:track:2XYkvc4UWMO9U2iQcIjJe7,Te Extraño - Bachata Version,[Xtreme],"[tropical, bachata, latin]",2,0.793,0.592,6,-4.749,...,0.0876,0.892,129.960,audio_features,2XYkvc4UWMO9U2iQcIjJe7,spotify:track:2XYkvc4UWMO9U2iQcIjJe7,https://api.spotify.com/v1/tracks/2XYkvc4UWMO9...,https://api.spotify.com/v1/audio-analysis/2XYk...,213973,4
970,"[spotify:artist:3rs3EOlJ8jyPpdGiQ9Mhub, spotif...",spotify:track:0wDEs6WvqDHq4XJZC0dHhO,Hoja En Blanco,"[Monchy & Alexandra, Alexandra]","[tropical, bachata, latin]",64,0.932,0.772,0,-3.599,...,0.1760,0.863,135.063,audio_features,0wDEs6WvqDHq4XJZC0dHhO,spotify:track:0wDEs6WvqDHq4XJZC0dHhO,https://api.spotify.com/v1/tracks/0wDEs6WvqDHq...,https://api.spotify.com/v1/audio-analysis/0wDE...,307200,4
971,[spotify:artist:5lwmRuXgjX8xIwlnauTZIP],spotify:track:6I86RF3odBlcuZA9Vfjzeq,Eres Mía,[Romeo Santos],"[tropical, bachata, latin]",72,0.843,0.729,6,-3.634,...,0.2060,0.903,123.046,audio_features,6I86RF3odBlcuZA9Vfjzeq,spotify:track:6I86RF3odBlcuZA9Vfjzeq,https://api.spotify.com/v1/tracks/6I86RF3odBlc...,https://api.spotify.com/v1/audio-analysis/6I86...,250640,4


In [8]:
#     playlists = sp.user_playlist_create(username, playlist_name,
#                                         description=playlist_description)
# results = sp.user_playlist_add_tracks(username, playlist_id, track_ids)